In [20]:
import xarray as xr
import numpy as np
import pandas as pd
from  scipy.io import loadmat
from scipy.linalg import block_diag
from CLM_vertical_utils import *
from CLM_vertical import *
from collections import namedtuple

# automatically reload external modules
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
# load the taus 
CLM_params = xr.open_dataset('../data/clm5_params.c171117.nc')
taus = np.array([CLM_params['tau_cwd'],CLM_params['tau_l1'],CLM_params['tau_l2_l3'],CLM_params['tau_l2_l3'],CLM_params['tau_s1'],CLM_params['tau_s2'],CLM_params['tau_s3']]).squeeze()
taus = taus * DAYS_PER_YEAR * SECS_PER_DAY

# load soil depth mat file 
fn = '../../../postdoc/projects/disordered_kinetics/gcb_matrix_supp_data/soildepth.mat'
mat = loadmat(fn)
zisoi = mat['zisoi'].squeeze()
zsoi = mat['zsoi'].squeeze()
dz = mat['dz'].squeeze()
dz_node = mat['dz_node'].squeeze()

# load gridded nc file with the inputs, initial values, and the environmental variables
global_da = xr.open_dataset('../data/global_demo_in.nc')

In [45]:
secspday = 86400
Gamma_soil = 1e-4 / (secspday * 365)
F_soil = 0

config = ConfigParams(decomp_depth_efolding=0.5, taus=taus, Gamma_soil=Gamma_soil, F_soil=F_soil,
                      zsoi=zsoi, zisoi=zisoi, dz=dz, dz_node=dz_node, nlevels=10, npools=7)
global_data = GlobalData(global_da)

ldd = global_data.make_ldd(global_da['LAT'][43].values, global_da['LON'][118].values)

CLM_model = CLM_vertical(config, ldd)

CLM_model.run(timesteps=range(12),dt = secspday*30)

/Users/yinonmb/Library/CloudStorage/OneDrive-CaliforniaInstituteofTechnology/git/soil_diskin/notebooks/CLM_vertical_utils.py:70: RuntimeWarning: invalid value encountered in divide
  Pe_e = F_e / D_e


<xarray.DataArray 'CWDC_VR' (TIME: 13, pooldepth: 70)>
array([[ 4.23001680e+04,  2.31345488e+04,  1.15746162e+04,
         5.42263770e+03,  2.45398877e+03,  1.18213574e+03,
         6.02106079e+02,  2.95110535e+02,  1.05446503e+02,
         8.80662155e+00,  1.81343399e+02,  4.70204239e+01,
         2.64490185e+01,  1.56547832e+01,  9.72735023e+00,
         6.36400127e+00,  3.98150587e+00,  2.21744704e+00,
         8.88987958e-01,  1.93034649e-01,  5.08808740e+03,
         1.69696606e+03,  8.11108948e+02,  4.06494385e+02,
         2.06778030e+02,  1.11388077e+02,  5.97003517e+01,
         2.98084087e+01,  1.08902597e+01,  1.51816165e+00,
         1.87024890e+03,  6.25801819e+02,  3.03810364e+02,
         1.56258331e+02,  8.22308121e+01,  4.55319672e+01,
         2.46850033e+01,  1.23786507e+01,  4.54673529e+00,
         6.86236382e-01,  1.21701501e+03,  9.27678406e+02,
         5.05097260e+02,  2.63754425e+02,  1.35847214e+02,
         7.38106384e+01,  3.91886368e+01,  1.92493515e+01,
         7.48710966e+00,  1.98455918e+00,  2.53172988e+04,
         2.23629395e+04,  1.50994287e+04,  8.09866943e+03,
         3.98889453e+03,  2.11525854e+03,  1.10910681e+03,
         5.34799438e+02,  2.17776749e+02,  7.53903809e+01,
...
         2.72342119e+05,  8.52495311e+04,  1.95131002e+04,
         2.97983600e+03,  3.16709968e+02,  2.83000831e+01,
         5.39095976e+00,  1.56509039e+00,  1.67997499e+04,
         1.77323704e+04,  7.86805459e+03,  3.56768958e+03,
         1.44465526e+03,  5.17649670e+02,  1.71039537e+02,
         5.60975585e+01,  1.85233411e+01,  4.21635978e+00,
         8.01759646e+03,  8.83698234e+03,  3.97784314e+03,
         1.78722725e+03,  7.14694999e+02,  2.51549092e+02,
         8.08613161e+01,  2.55792244e+01,  8.36792982e+00,
         2.03554380e+00, -9.79343630e+04, -5.52623802e+05,
        -2.05021979e+05, -6.41390451e+04, -1.45890194e+04,
        -2.13888634e+03, -1.74812903e+02,  7.75241552e+00,
         6.86041147e+00,  1.96443279e+00,  5.41291669e+04,
         1.28081134e+05,  5.47145669e+04,  2.04190291e+04,
         6.73616491e+03,  2.48875663e+03,  1.13213670e+03,
         5.34434354e+02,  2.17697165e+02,  7.53876507e+01,
         1.64375952e+04,  1.68218165e+04,  1.54957616e+04,
         1.34816281e+04,  1.05493924e+04,  7.21947283e+03,
         4.38674822e+03,  2.60795778e+03,  1.85292255e+03,
         1.67754189e+03]])
Coordinates:
    LON          float32 295.0
    LAT          float32 -8.526
    TIME1        object 0001-02-01 00:00:00
  * pooldepth    (pooldepth) object MultiIndex
  * pools        (pooldepth) int64 0 0 0 0 0 0 0 0 0 0 1 ... 6 6 6 6 6 6 6 6 6 6
  * LEVDCMP1_10  (pooldepth) float64 0.007101 0.02792 0.06226 ... 1.728 2.865
Dimensions without coordinates: TIME
Attributes:
    long_name:  CWDC_EMULATOR_VR[L=1]
    history:    From globe_matlab_280ppm_sub

<xarray.DataArray 'TIME' (TIME: 12)>
array([cftime.DatetimeNoLeap(1, 2, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 3, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 4, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 5, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 6, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 7, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 8, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 9, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 10, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 11, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 12, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2, 1, 1, 0, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * TIME     (TIME) object 0001-02-01 00:00:00 ... 0002-01-01 00:00:00
Attributes:
    long_name:      time
    bounds:         TIME_bnds
    axis:           T
    point_spacing:  uneven
    standard_name:  time

In [ ]:

# unpack the environmental variables
sand_da = global_da['CELLSAND']
w_scalar_da = global_da['W_SCALAR']
t_scalar_da = global_da['T_SCALAR']
o_scalar_da = global_da['O_SCALAR']
n_scalar_da = global_da['FPI_VR']

# upack the initial values
CWD = global_da['CWDC_VR']
LITR1 = global_da['LITR1C_VR']
LITR2 = global_da['LITR2C_VR']
LITR3 = global_da['LITR3C_VR']
SOIL1 = global_da['SOIL1C_VR']
SOIL2 = global_da['SOIL2C_VR']
SOIL3 = global_da['SOIL3C_VR']

# unpack the inputs
CWD_input = global_da['TOTC2CWDC_VR']
LITR1_input = global_da['TOTC2LITRMETC_VR']
LITR2_input = global_da['TOTC2LITRCELC_VR']
LITR3_input = global_da['TOTC2LITRLIGC_VR']
zero_da = xr.zeros_like(LITR1_input)
inputs = xr.concat([CWD_input, LITR1_input, LITR2_input, LITR3_input, zero_da, zero_da, zero_da], dim='pools')
# inputs/inputs.sum(dim=['pools','LEVDCMP1_10'])

In [45]:
global_data.n_scalar_da

<xarray.DataArray 'FPI_VR' (TIME1: 12, LEVDCMP1_10: 10, LAT: 96, LON: 144)>
[1658880 values with dtype=float32]
Coordinates:
  * LON          (LON) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * LAT          (LAT) float32 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
  * LEVDCMP1_10  (LEVDCMP1_10) float32 0.007101 0.02792 0.06226 ... 1.728 2.865
  * TIME1        (TIME1) object 0001-02-01 00:00:00 ... 0002-01-01 00:00:00
Attributes:
    long_name:  fraction of potential immobilization
    units:      proportion
    history:    From globe_matlab_280ppm

Diffusion advection module

In [140]:
def shift(arr, num,direction):
    """
    Shift an array by a given number of elements in a given direction

    Parameters
    ----------
    arr : np.array
        Array to shift
    num : int
        Number of elements to shift by
    direction : int
        Direction to shift in. 1 for right, -1 for left

    Returns
    -------
    np.array
        Shifted array
    """
    
    assert direction in ['right','left']
    assert num >0 
    if direction=='right':
        return np.pad(arr,(num,0),mode='constant')[:-num]
    else:
        return np.pad(arr,(0,num),mode='constant')[num:]


In [46]:
global_da['CELLSAND'][0,:].sel(LAT=25,LON=50,method='nearest')

<xarray.DataArray 'CELLSAND' (LEVDCMP1_10: 10)>
[10 values with dtype=float32]
Coordinates:
    LON          float32 50.0
    LAT          float32 25.58
  * LEVDCMP1_10  (LEVDCMP1_10) float32 0.007101 0.02792 0.06226 ... 1.728 2.865
    TIME         object 0001-02-01 00:00:00
Attributes:
    long_name:  Percentage that is sand
    units:      percent
    history:    From globe_matlab_280ppm

In [ ]:
# w_e = (shift(zisoi,1,'right') - shift(zsoi,1,'right')) / dz_node
nlevels = 10;
npools = 7;
sand_content = global_da['CELLSAND'][0,:].sel(LAT=25,LON=50,method='nearest').values # choose a random location
assert len(sand_content) == nlevels

# The functional parameterization of the transfer coefficients is based on # this is based on the
# original CENTURY model parameterization in Parton et al. 1998 (https://link.springer.com/article/10.1007/BF02180320) in Figure 1

# t is a number dependent on the sand content that determines the transfer coefficient fraction of carbon that is lost to respiration
t = 0.85 - 0.68 * 0.01 * (100 - sand_content)

# f is the fraction of carbon from a specific pool that is transferred to another pool
f_s1s2 = 1 - 0.004 / (1 - t)
f_s1s3 = 0.004 / (1 - t)
f_s2s1 = 0.42 / 0.45 * np.ones(nlevels)
f_s2s3 = 0.03 / 0.45 * np.ones(nlevels)

# rf is the fractio of carbon in a specific flux between pools that is lost to respiration (1-CUE)
rf_s1s2 = t
rf_s1s3 = t

# Using the formalism a_i,j = (1-rf_i,j) * f_i,j, where a_i,j are the coefficients in the A matrix 
# Implementation accroding to Eq. 3 in Huang et al. 2017 (https://onlinelibrary.wiley.com/doi/10.1111/gcb.13948)
Adiag = -np.eye(nlevels) #A11-A77
A_zero = np.zeros((nlevels,nlevels))
A31 = 0.76 * np.eye(nlevels) # CWD -> Litter2
A41 = 0.24 * np.eye(nlevels) # CWD -> Litter3
A52 = (1-0.55) * np.eye(nlevels) # Litter1 -> SOM2
A53 = (1-0.5) * np.eye(nlevels) # Litter2 -> SOM1
A56 = np.diag((1-0.55) * f_s2s1) # SOM1 -> SOM3
A57 = (1-0.55) * np.eye(nlevels) # SOM3 -> SOM1
A64 = (1-0.5) * np.eye(nlevels) # Litter3 -> SOM2
A65 = np.diag((1 - rf_s1s2) * f_s1s2) # SOM1 -> SOM2
A75 = np.diag((1 - rf_s1s3) * f_s1s3) # SOM1 -> SOM3
A76 = np.diag((1-0.55) * f_s2s3) # SOM2 -> SOM3

A_matrix = np.block([
    [Adiag     , A_zero    , A_zero    , A_zero    , A_zero    , A_zero    , A_zero    ],
    [A_zero    , Adiag     , A_zero    , A_zero    , A_zero    , A_zero    , A_zero    ],
    [A31       , A_zero    , Adiag     , A_zero    , A_zero    , A_zero    , A_zero    ],
    [A41       , A_zero    , A_zero    , Adiag     , A_zero    , A_zero    , A_zero    ],
    [A_zero    , A52       , A53       , A_zero    , Adiag     , A56       , A57       ],
    [A_zero    , A_zero    , A_zero    , A64       , A65       , Adiag     , A_zero    ],
    [A_zero    , A_zero    , A_zero    , A_zero    , A75       , A76       , Adiag     ]
         ])




(15,)

In [97]:
# w_scalar = global_demo_ds['W_SCALAR'][0,:].sel(LAT=25,LON=50,method='nearest').values # choose a random location
# t_scalar = global_demo_ds['T_SCALAR'][0,:].sel(LAT=25,LON=50,method='nearest').values # choose a random location
# o_scalar = global_demo_ds['O_SCALAR'][0,:].sel(LAT=25,LON=50,method='nearest').values # choose a random location
# n_scalar = global_demo_ds['FPI_VR'][0,:].sel(LAT=25,LON=50,method='nearest').values # choose a random location

example_data = loadmat('test_examples/K_matrix_positive_examples/test_K_mat_5.mat')
inputs = example_data['example'].squeeze()
decomp_depth_efolding = example_data['decomp_depth_efolding'].squeeze()
print(decomp_depth_efolding)
w_scalar = inputs[0,:]
t_scalar = inputs[1,:]
o_scalar = inputs[2,:]
n_scalar = inputs[3,:]

CLM_params = xr.open_dataset('../data/clm5_params.c171117.nc')
CLM_params['tau_l1'].values
taus = np.array([CLM_params['tau_cwd'],CLM_params['tau_l1'],CLM_params['tau_l2_l3'],CLM_params['tau_l2_l3'],CLM_params['tau_s1'],CLM_params['tau_s2'],CLM_params['tau_s3']]).squeeze()
days_per_year = 365
decomp_depth_efolding = 0.5


# calculate k's from tau's
ks = 1 / (secspday * days_per_year * taus)
depth_scalar = np.exp(-zsoi[:nlevels]/decomp_depth_efolding)
k_modifier = (t_scalar * w_scalar * o_scalar * depth_scalar)
K_matrix = block_diag(*[np.diag(k * k_modifier * n_scalar if i in [1,2,3] else k * k_modifier) for i,k in enumerate(ks)])
K_matrix


0.5


array([[8.44091709e-10, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 8.09658390e-10, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 7.55927348e-10, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.61075528e-12, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 4.05549098e-13, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 4.17332569e-14]])

In [89]:
ks

array([9.51293782e-09, 5.86631137e-07, 1.55377987e-07, 1.55377987e-07,
       2.31481483e-07, 6.34195840e-09, 1.42694060e-10])

In [5]:
# w_e = (shift(zisoi,1,'right') - shift(zsoi,1,'right')) / dz_node
secspday = 86400;
Gamma_soil = 1e-4 / (secspday * 365);
F_soil= 0;
nlevels = 10;
npools = 7;


# A function from Patankar, Table 5.2, pg 95
aaa = np.vectorize(lambda pe: np.max ([0, (1 - 0.1 * np.abs(pe))**5]))

Gamma_vec = np.ones(nlevels+1)*Gamma_soil
F_vec = np.ones(nlevels+1)*F_soil

# Calculate the weighting between lfactors for the diffusion and advection terms
w_e = np.zeros(nlevels+1)
w_e[1:] = (zisoi[:nlevels] - zsoi[:nlevels]) / dz_node[1:nlevels+1]
Gamma_e = np.zeros(nlevels+1)
Gamma_e[1:] = 1 / ((1 - w_e[1:nlevels+1]) / Gamma_vec[1:nlevels+1] + w_e[1:nlevels+1] / Gamma_vec[nlevels]); # Harmonic mean of diffus

# Calculate the weighting between lfactors for the diffusion and advection terms
w_p = np.zeros(nlevels+1)
w_p[:nlevels] = (zsoi[1:nlevels+1] - zisoi[:nlevels]) / dz_node[1:nlevels+1]
Gamma_p = np.zeros(nlevels+1)
Gamma_p[:nlevels] = 1 / ((1 - w_p[:nlevels]) / Gamma_vec[:nlevels] + w_p[:nlevels] / Gamma_vec[1:nlevels+1]); # Harmonic mean of diffus

## TODO - pop the above code into a separate function and compare againt the output from the matlab code

# Define the D and F values for each layer the according to Eq. 5.9 in Patankar, pg. 82
D_e = Gamma_e / dz_node[:nlevels+1]
D_p = np.zeros(nlevels+1)
D_p[:nlevels] = Gamma_p[:nlevels] / dz_node[1:nlevels+1]
D_p[-1] = D_e[-1]
F_e = F_vec
F_p = F_vec
F_p[-1] = 0


# Define the Peclet number
Pe_e = F_e / D_e
Pe_e[0] = 0;
Pe_p = F_p / D_p

# Define the vectors for the tridiagonal matrix
a_tri_e =  -( D_e * aaa(Pe_e) + np.max([F_e, np.zeros(nlevels+1)],axis=0))
c_tri_e =  - (D_p * aaa(Pe_p) + np.max([-F_p, np.zeros(nlevels+1)],axis=0))
b_tri_e = - a_tri_e - c_tri_e

# Define the upper and lower bounaries
b_tri_e[0] = - c_tri_e[0]
b_tri_e[-2] = - a_tri_e[-2]

# Define the tridiagonal matrix
tri_matrix = np.diag(a_tri_e[:-1],k=-1)[1:,1:] + np.diag(b_tri_e[:-1],k=0) + np.diag(c_tri_e[:-1],k=1)[:-1,:-1]

# Define the block diagonal matrix
tri_matrix = block_diag(*[tri_matrix]*npools)

# Set the first pool to zero
tri_matrix[:nlevels,:nlevels] = 0

# Divide the matrix by dz
tri_matrix = (tri_matrix.T/np.tile(dz[:nlevels],npools)).T

/var/folders/6f/_716m82x3n32hwr7x9h4fp2c0000gp/T/ipykernel_37536/3736361782.py:40: RuntimeWarning: invalid value encountered in divide
  Pe_e = F_e / D_e


In [ ]:
tri_matrix

A function to build the tridiagonal matrix for the diffusion advection equation.

In [ ]:
def tri_matrix(Gamma_soil,F_soil,npools,nlevels,dz,dz_node,zsoi,zisoi):
    """
    Create a tridiagonal matrix for soil carbon pools

    Parameters
    ----------
    Gamma_soil : float
        Diffusion coefficient
    F_soil : float
        Advection coefficient
    npools : int
        Number of soil carbon pools
    nlevels : int
        Number of soil layers
    dz : np.array
        Thickness of soil layers (m)
    dz_node : np.array
        Distance between layer interfaces (m)
    zsoi : np.array
        Depth of soil layers (m)
    zisoi : np.array
        Depth of soil layer interfaces (m)

    Returns
    -------
    np.array
        Tridiagonal matrix
    """

    # A function from Patankar, Table 5.2, pg 95
    aaa = np.vectorize(lambda pe: np.max ([0, (1 - 0.1 * np.abs(pe))**5]))

    Gamma_vec = np.ones(nlevels+1)*Gamma_soil
    F_vec = np.ones(nlevels+1)*F_soil

    # Calculate the weighting between lfactors for the diffusion and advection terms
    w_e = np.zeros(nlevels+1)
    w_e[1:] = (zisoi[:nlevels] - zsoi[:nlevels]) / dz_node[1:nlevels+1]
    Gamma_e = np.zeros(nlevels+1)
    Gamma_e[1:] = 1 / ((1 - w_e[1:nlevels+1]) / Gamma_vec[1:nlevels+1] + w_e[1:nlevels+1] / Gamma_vec[nlevels]); # Harmonic mean of diffus

    # Calculate the weighting between lfactors for the diffusion and advection terms
    w_p = np.zeros(nlevels+1)
    w_p[:nlevels] = (zsoi[1:nlevels+1] - zisoi[:nlevels]) / dz_node[1:nlevels+1]
    Gamma_p = np.zeros(nlevels+1)
    Gamma_p[:nlevels] = 1 / ((1 - w_p[:nlevels]) / Gamma_vec[:nlevels] + w_p[:nlevels] / Gamma_vec[1:nlevels+1]); # Harmonic mean of diffus

    ## TODO - pop the above code into a separate function and compare againt the output from the matlab code

    # Define the D and F values for each layer the according to Eq. 5.9 in Patankar, pg. 82
    D_e = Gamma_e / dz_node[:nlevels+1]
    D_p = np.zeros(nlevels+1)
    D_p[:nlevels] = Gamma_p[:nlevels] / dz_node[1:nlevels+1]
    D_p[-1] = D_e[-1]
    F_e = F_vec
    F_p = F_vec
    F_p[-1] = 0


    # Define the Peclet number
    Pe_e = F_e / D_e
    Pe_e[0] = 0;
    Pe_p = F_p / D_p

    # Define the vectors for the tridiagonal matrix
    a_tri_e =  -( D_e * aaa(Pe_e) + np.max([F_e, np.zeros(nlevels+1)],axis=0))
    c_tri_e =  - (D_p * aaa(Pe_p) + np.max([-F_p, np.zeros(nlevels+1)],axis=0))
    b_tri_e = - a_tri_e - c_tri_e

    # Define the upper and lower bounaries
    b_tri_e[0] = - c_tri_e[0]
    b_tri_e[-2] = - a_tri_e[-2]

    # Define the tridiagonal matrix
    tri_matrix = np.diag(a_tri_e[:-1],k=-1)[1:,1:] + np.diag(b_tri_e[:-1],k=0) + np.diag(c_tri_e[:-1],k=1)[:-1,:-1]

    # Define the block diagonal matrix
    tri_matrix = block_diag(*[tri_matrix]*npools)
    
    # Set the first pool to zero
    tri_matrix[:nlevels,:nlevels] = 0

    # Divide the matrix by dz
    tri_matrix = (tri_matrix.T/np.tile(dz[:nlevels],npools)).T

    return tri_matrix